# LangSmith 데이터셋 생성

자체 RAG 평가용 데이터셋을 구축하는 방법에 대해 알아보겠습니다.

먼저, 데이터셋 구축을 위해서는 크게 3가지 과정에 대한 이해가 필요합니다.

Case: Retrieval 이 Question 에 Relevant 한지 평가

> Question - Retrieval

![](./assets/eval-03.png)

Case: Answer 이 Question 에 Relevant 한지 평가

> Question - Answer   

![](./assets/eval-04.png)

Case: Answer 가 Retrieval 된 문서 안에서 답변하였는지 (Hallucination Check)

> Retrieval - Answer

![](./assets/eval-05.png)

따라서, `Question`, `Retrieval`, `Answer` 의 3가지 정보가 필요한 것이 일반적이지만, `Retrieval` 에 대한 Ground Truth 구축이 사실상 어렵습니다.

만약, `Retrieval` 에 대한 Ground Truth 가 존재한다면, 모두 데이터셋으로 저장하여 활용하고, 그렇지 않다면 `Question`, `Answer` 만으로 데이터셋을 구축하여 활용할 수 있습니다.

In [ ]:
# 설치
# !pip install -qU langsmith langchain-teddynote

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("금융리포트RAG-Evaluations")

LangSmith 추적을 시작합니다.
[프로젝트명]
금융리포트RAG-Evaluations


## 데이터셋 생성

`inputs` 와 `outputs` 를 활용하여 데이터셋을 생성합니다.

데이터셋은 `question` 과 `answer` 로 구성됩니다.

In [8]:
import pandas as pd

# 질문과 답변 목록
inputs = [
    "주 초반 국내 증시는 상승과 하락 중 어디로 출발하고 있나?",
    "미국은 일본에 대한 상호관세율을 기존 제안에서 몇 % 인하하였나?",
    "일본은 관세율 인하 조건으로 미국에 투자를 얼마나 하기로 하였나?",
    "강세를 보인 업종을 나열해줘.",
    "강세를 보인 업종들 상승 수치를 합산하면? 수치로 답변해줘",
    "한국 증시에 끼치는 주요 영향 요소에 대해 나열해줘",
    "지난주 국내 증시의 변동성 확대 주요 원인은? ",
    "지난주 동향 중 국제유가는 상승이였나? 하락이였나?",
    "지난주 동향 중 금은 상승이였나? 하락이였나?",
    "GLD 주가 및 누적 자금유입 추세 차트에서 급격한 상승이 시작되는 구간은?",
    "2025년 7월 28일 업종별 외국인 주간 누적 순매수 차트에서 외국인 투자자들이 가장 선호하는 업종은 무엇인가요? 종목만 답변해줘",
    "기관 및 외국인 누적 순매수 추이 차트에서 외국인의 매도세가 강했던 시기는 언제인가요?",
    "주요 상품 가격 및 변동률에서 천연가스의 1주일 변동률은 어떻게 되나요? 수치로 답변해줘",
    "해외 채권 금리 현황에서 1주 변화량이 가장 큰 종목은? 종목만 답변해줘",
    "금 ETF 및 관련 자산 현황에서 은 ETF(SLV)의 1년 수익률은 얼마인가요? 수치로 답변해줘",
    "2025년 7월 28일 글로벌 ETF 주간 리포트 주간 수익률이 가장 낮은 ETF는 무엇인가요? 종목만 답변해줘",
    "2025년 7월 28일 글로벌 ETF 주간 리포트 이번 주 가장 높은 수익률을 기록한 ETF는 무엇인가요? 종목만 답변해줘",
]

# 질문에 대한 답변 목록
outputs = [
    "상승 출발하고 있습니다.",
    "10% 인하하였습니다.",
    "5,500억 달러 투자",
    "강세 종목은 전기/가스, 운송장비/부품, 기계/장비, 금속, 화학",
    "16.64%",
    "한미 상호관세 협상, 7월 FOMC, 7월 미국 ISM 제조업 PMI 및 비농업 고용, 미국 M7 실적, 국내 기업 실적, 국내 세제 개편안",
    "미국과 한국 회담 돌연 취소",
    "지난주 동향에서 국제유가는 하락이었습니다.",
    "지난주 동향에서 금은 하락이었습니다.",
    "2024년 9월과 2025년 4월 사이",
    "제조업",
    "2022년 7월경과 2025년 5월경",
    "-12.76%",
    "일본 국채 10년",
    "31.64%",
    "AdvisorShares Pure US Cannabis ETF",
    "VanEck Rare Earth & Strat Mts ETF",
]

# 질문과 답변 쌍 생성
qa_pairs = [{"question": q, "answer": a} for q, a in zip(inputs, outputs)]

# 데이터프레임으로 변환
df = pd.DataFrame(qa_pairs)

# 데이터프레임 출력
df.head()

,question,answer
0,주 초반 국내 증시는 상승과 하락 중 어디로 출발하고 있나?,상승 출발하고 있습니다.
1,미국은 일본에 대한 상호관세율을 기존 제안에서 몇 % 인하하였나?,10% 인하하였습니다.
2,일본은 관세율 인하 조건으로 미국에 투자를 얼마나 하기로 하였나?,"5,500억 달러 투자"
3,강세를 보인 업종을 나열해줘.,"강세 종목은 전기/가스, 운송장비/부품, 기계/장비, 금속, 화학"
4,강세를 보인 업종들 상승 수치를 합산하면? 수치로 답변해줘,16.64%


혹은 이전의 튜토리얼에서 생성한 Synthetic Dataset 을 활용할 수 있습니다.

아래 코드는 업로드한 HuggingFace Dataset 을 활용하는 예시입니다.

(참고) 아래의 주석을 풀고 실행하여 `datasets` 라이브러리를 업데이트 후 진행해 주세요. 

In [ ]:
# !pip install -qU datasets

## LangSmith 테스트를 위한 Dataset 생성

- `Datasets & Testing` 에 새로운 데이터셋을 생성합니다.

![](./assets/eval-06.png)

csv 파일에서 LangSmith UI를 사용하여 직접 데이터셋을 생성할 수도 있습니다.

자세한 사항은 아래 문서를 참고해 주세요.

- [LangSmith UI 문서](https://docs.smith.langchain.com/evaluation/faq/manage-datasets)

In [9]:
from langsmith import Client

client = Client()
dataset_name = "RAG_EVAL_(Financial Reports)_DATASET"


# 데이터셋 생성 함수
def create_dataset(client, dataset_name, description=None):
    for dataset in client.list_datasets():
        if dataset.name == dataset_name:
            return dataset

    dataset = client.create_dataset(
        dataset_name=dataset_name,
        description=description,
    )
    return dataset


# 데이터셋 생성
dataset = create_dataset(client, dataset_name)

# 생성된 데이터셋에 예제 추가
client.create_examples(
    inputs=[{"question": q} for q in df["question"].tolist()],
    outputs=[{"answer": a} for a in df["answer"].tolist()],
    dataset_id=dataset.id,
)

{'example_ids': ['f75dca30-bad0-42c5-b064-e8f4793747b4',
  '0f089a8b-254e-41e2-8686-84c5e52622e9',
  '583c9664-7024-4975-8799-2717c610c8f3',
  '9ed033d1-10a9-4be3-8bad-b88d1d223a03',
  'd0a217c0-47c0-493a-ad47-b61a51a5e5d7',
  'a93731ef-7ded-46c4-bbad-46883e5f8382',
  '5ee19e02-5cfc-452e-b699-567f6ca9eab1',
  'd0ab8f99-2691-4ec5-ba7b-414ebe536658',
  'bfbbb777-863b-434d-9cdd-f73e4060cb6c',
  '05ec8bc0-cb28-4830-a747-5d9cba1a19d0',
  '0e208d51-26c6-4e07-a978-4db4861aaf66',
  'a55eda4e-95f8-41bc-b6d8-9e47d6d3c93c',
  '8664f605-8f9e-42be-86c9-79866c6e9b3c',
  '6a296dcf-552d-465c-9fbc-337f0ccb2ed0',
  '3859f172-89d9-436f-82ef-651b6590e835',
  '8a6874ed-86b0-4360-9048-c965b030859f',
  'a4eadcb7-77fd-498d-9652-731c547313db'],
 'count': 17}

데이터셋에 예제를 나중에 추가할 수 있습니다.

In [ ]:
# # 새로운 질문 목록
# new_questions = [
#     "삼성전자가 만든 생성형 AI의 이름은 무엇인가요?",
#     "구글이 테디노트에게 20억달러를 투자한 것이 사실입니까?",
# ]

# # 새로운 답변 목록
# new_answers = [
#     "삼성전자가 만든 생성형 AI의 이름은 테디노트 입니다.",
#     "사실이 아닙니다. 구글은 앤스로픽에 최대 20억 달러를 투자하기로 합의했으며, 이 중 5억 달러를 우선 투자하고 향후 15억 달러를 추가로 투자하기로 했습니다.",
# ]

# # UI에서 업데이트된 버전 확인
# client.create_examples(
#     inputs=[{"question": q} for q in new_questions],
#     outputs=[{"answer": a} for a in new_answers],
#     dataset_id=dataset.id,
# )

축하합니다! 이제 데이터셋이 준비되었습니다.